<a href="https://colab.research.google.com/github/m-wessler/oper-scrape/blob/main/AFD_Word_Scrape_weather_gov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import requests
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

# Define the search terms and precision terms
search_terms = [
    "GFS", "GFS ENSEMBLE", "GEFS", "ECMWF", "ECMWF ENSEMBLE",
    "EPS", "HRRR", "HREF", "ENSEMBLE", "EFI", "NBM",
    "NATIONAL BLEND", "CLUSTER"
]

precision_terms = [
    "PERCENTILE", "PROBABILITY", "POSSIBLE", "EXPECTED",
    "CHANCE", "LIKELY"
]

# List of WFO sites
WFO = [
    "BYZ", "BOI", "LKN", "EKA", "FGZ", "GGW", "TFX", "VEF", "LOX", "MFR",
    "MSO", "PDT", "PSR", "PIH", "PQR", "REV", "STO", "SLC", "SGX", "MTR",
    "HNX", "SEW", "OTX", "TWC"
]

# Initialize counters
wfo_search_term_counts = {wfo: {term: 0 for term in search_terms} for wfo in WFO}
wfo_precision_term_counts = {wfo: {term: 0 for term in precision_terms} for wfo in WFO}

# Iterate over each WFO site
for wfo in tqdm(WFO, desc='WFO Sites'):
    # Iterate over the most recent 15 versions
    for version in tqdm(range(1, 16), desc=f'Versions for {wfo}', leave=False):
        # Fetch the content from the URL
        url = f'https://forecast.weather.gov/product.php?site=NWS&issuedby={wfo}&product=AFD&format=CI&version={version}&glossary=1'
        try:
            response = requests.get(url)
            response.raise_for_status()
            content = response.text

            # Parse the HTML content to get the text
            soup = BeautifulSoup(content, 'html.parser')
            text_content = soup.get_text()

            # Count occurrences of each term using regex with word boundaries
            for term in search_terms:
                wfo_search_term_counts[wfo][term] += len(re.findall(r'\b' + re.escape(term) + r'\b', text_content, re.IGNORECASE))

            for term in precision_terms:
                wfo_precision_term_counts[wfo][term] += len(re.findall(r'\b' + re.escape(term) + r'\b', text_content, re.IGNORECASE))
        except requests.exceptions.RequestException as e:
            print(f"Failed to retrieve version {version} for WFO {wfo}: {e}")

# Combine the counts into a single DataFrame
search_term_df = pd.DataFrame(wfo_search_term_counts).T
precision_term_df = pd.DataFrame(wfo_precision_term_counts).T
combined_df = pd.concat([search_term_df, precision_term_df], axis=1, keys=['Search Terms', 'Precision Terms'])

combined_df.to_csv('./test.csv')

# Print the combined DataFrame
print("Combined Term Counts:")
combined_df

In [ ]:
combined_df

Search Terms                                                       \
             GFS GFS ENSEMBLE GEFS ECMWF ECMWF ENSEMBLE EPS HRRR HREF   
BYZ           20            0    0     5              0   0    2    0   
BOI            2            0    0     2              0   0    0    0   
LKN            4            0    0     1              0   0    0    0   
EKA            4            0    0     7              3   0    1   18   
FGZ            0            0    0     0              0   0    0    5   
GGW            2            0    0     0              0   0    1    3   
TFX            0            0    0     0              0   0    0    0   
VEF            0            0    0     0              0   4    0    2   
LOX            0            0    0     0              0   2    6    2   
MFR            0            0    0     0              0   0    0    0   
MSO            1            0    0     0              0   0    1    0   
PDT            5            3    0    16              0   0    0    0   
PSR            0            0    3     0              0   3    0    6   
PIH            4            0    0     4              0   0    0    0   
PQR           10            0    0    10              0   0    0    7   
REV            0            0    0     0              0   0    0    1   
STO            0            0    0     0              0   0    0    0   
SLC            0            0    0     0              0   0    0    0   
SGX            0            0    0     0              0   0    0    2   
MTR            0            0    0     0              0   0    2    0   
HNX            0            0    0     0              0   0    0    0   
SEW            0            0    0     0              0   0    0    0   
OTX            9            0    6     6              3   0    0   13   
TWC            2            0    0     5              0   0    0    4   

                                            Precision Terms              \
    ENSEMBLE EFI NBM NATIONAL BLEND CLUSTER      PERCENTILE PROBABILITY   
BYZ        6   0  11              3       1               4           3   
BOI        2   0   0              0       0               0           2   
LKN        0   0   0              0       0               0           0   
EKA       26   3   8              0       0               2           5   
FGZ        0   0   0              0       0               0           0   
GGW        0   0  15              0       0               6           2   
TFX       11   0   4              0       0               0          19   
VEF        5   0  16              0       9               0           4   
LOX        6   0   2              0       0               0           0   
MFR        0   0   0              0       0               0           0   
MSO        8   0   0              0       0               0           3   
PDT       25  15  31              0       1               0           7   
PSR       36   0   3              0       3               3           0   
PIH        0   0   0              1       5               1           1   
PQR       40   0  26              0       0               0          18   
REV        5   0   2              0       0               0           0   
STO       25   2  10              6       6               0          29   
SLC        7   1   9              0       1               2           0   
SGX        0   0   6              0       2               0           0   
MTR       12   0   2              0       3               0           9   
HNX        3   0   2              1       1               2          35   
SEW        7   0   0              0       0               0           1   
OTX       15   0  21              0       0               3           4   
TWC        6   1   0              0       0               3           0   

                                     
    POSSIBLE EXPECTED CHANCE LIKELY  
BYZ       35       66     94     15  
BOI       34       26     74      4  
LK